### SGD 방식 구현

* 랜덤하게 데이터를 섞기 위한 함수

  * torch.randperm(n) : 0 ~ n -1 까지의 정수를 랜덤하게 섞어서, 순열(배열)을 만들어 줌
  * torch.index_select(텐서 객체, 차원번호, 인덱스 텐서)

    * 차원번호는 예를 들어, |x| = (3,4)에서 0차원에 해당하는 값은 3 (행으로 이해하면 됨), 1차원에 해당하는 값은 4 (열로 이해하면 됨)
  
  * 특정 차원의 나열된 인덱스 번호 순서대로, 데이터를 섞어줌

In [2]:
import torch

data1 = torch.randn(3, 4)
print(data1)
indices = torch.tensor([1, 2])
print(indices)
print(torch.index_select(data1, 0, indices))
print(torch.index_select(data1, 1, indices))

tensor([[ 0.8158, -0.4771,  1.1792,  1.8413],
        [ 0.4434,  1.3455,  0.0997, -1.4989],
        [-1.5257, -0.9818,  1.4829, -0.3548]])
tensor([1, 2])
tensor([[ 0.4434,  1.3455,  0.0997, -1.4989],
        [-1.5257, -0.9818,  1.4829, -0.3548]])
tensor([[-0.4771,  1.1792],
        [ 1.3455,  0.0997],
        [-0.9818,  1.4829]])


### 테스트 구현

In [3]:
x = torch.ones(5000, 10) # 10개의 feature 가 있는 5000개의 데이터
y = torch.zeros(5000, 1) # 5000개의 데이터에 대한 실제 신경망에서 예측해야 하는 결과 값
learning_rate = 0.01 # 학습률
nb_epochs = 1000 # 전체 데이터 셋에 대해 학습을 몇 번 반복할지
minibatch_size = 256 # 미니배치 크기, 1epoch당 5000/256 = 약 20번의 미니배치 학습

In [4]:
import torch.nn as nn

input_dim = x.size(-1)
output_dim = y.size(-1)

# 보통 hidden layer는 출력에 가까울 수록 작아지게 설계하는 것이 일반적임 (더 좋은 성능)
model = nn.Sequential(
  nn.Linear(input_dim, 10),
  nn.LeakyReLU(0, 1),
  nn.Linear(10, 8),
  nn.LeakyReLU(0, 1),
  nn.Linear(8, 6),
  nn.LeakyReLU(0, 1),
  nn.Linear(6, output_dim)
)

loss_function = nn.MSELoss() # 평균 제곱 오차 손실 함수
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # 확률적 경사 하강법

In [9]:
indices = torch.randperm(x.size(0))
print(indices)
x_batch_list = torch.index_select(x, 0 , index = indices) # shuffle 된 데이터 셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언
y_batch_list = torch.index_select(y, 0 , index = indices) # shuffle 된 데이터 셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언
x_batch_list = x_batch_list.split(minibatch_size, dim = 0)
y_batch_list = y_batch_list.split(minibatch_size, dim = 0)
print(len(x_batch_list), len(y_batch_list))

tensor([1812,  703, 4288,  ..., 3306,  638,  274])
20 20


In [12]:
print(type(x_batch_list))
print(type(x_batch_list[0]))

<class 'tuple'>
<class 'torch.Tensor'>


### 참고 : 파이썬 zip() 내장 함수

* for 구문에서 두 개 이상의 리스트 변수를 같은 인덱스 번호의 데이터 별로 묶어서, 튜플로 반환할때 많이 활용하는 함수

In [14]:
# 학습

for index in range(nb_epochs):
  indices = torch.randperm(x.size(0))

  x_batch_list = torch.index_select(x, 0, index = indices) # shuffle 된 데이터 셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언
  y_batch_list = torch.index_select(y, 0, index = indices) # shuffle 된 데이터 셋으로, 데이터양이 상당하므로, 미니배치 변수로 선언
  x_batch_list = x_batch_list.split(minibatch_size, dim = 0)
  y_batch_list = y_batch_list.split(minibatch_size, dim = 0)

  for x_mini_batch, y_mini_batch in zip(x_batch_list, y_batch_list):
    y_mini_batch_pred = model(x_mini_batch)
    loss = loss_function(y_mini_batch_pred, y_mini_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss)
for param in model.parameters():
  print(param)

tensor(7.9936e-15, grad_fn=<MseLossBackward0>)
Parameter containing:
tensor([[ 0.1446, -0.0679, -0.1205, -0.1150,  0.0881,  0.2604, -0.2700,  0.0330,
         -0.2710,  0.0200],
        [-0.0127,  0.1181, -0.1243, -0.1206, -0.2893,  0.0285,  0.1410,  0.1030,
          0.1040, -0.1206],
        [ 0.1393,  0.2504, -0.1547,  0.2920,  0.2206, -0.2007, -0.2297,  0.0973,
          0.2636,  0.2649],
        [-0.2881, -0.1336,  0.1357,  0.1441,  0.0785,  0.2599, -0.2804, -0.1032,
         -0.3012,  0.0241],
        [-0.0588,  0.0343,  0.2265, -0.1741, -0.1457,  0.1260,  0.1580, -0.1585,
         -0.2403,  0.1517],
        [ 0.1539,  0.2652, -0.1648,  0.0897, -0.0022,  0.2849, -0.0790, -0.0369,
          0.0029,  0.1334],
        [-0.2463, -0.2889, -0.1888, -0.0933, -0.2540,  0.2675,  0.2638, -0.1787,
          0.2864, -0.2052],
        [ 0.2770, -0.0525, -0.2061,  0.2006,  0.0943, -0.0519,  0.2293,  0.2090,
         -0.2904,  0.2953],
        [ 0.1382, -0.1844,  0.2903, -0.3157, -0.1199,  0.06